<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF-280 - Estadística Computacional </h1>
    <h2> Probabilidad y Variables Aleatorias </h2>
    <h4> Diego Quezada </h4>
</center>

## Contenidos

* [Introducción](#intro)
* [Reglamento](#rules)
* [Experiencia](#experience)
    * [Fundamentales](#fundamentals)
    * [Estimación de pi](#pi)
    * [Integración de Monte Carlo](#montecarlo)
    * [Caminos aleatorios](#randomwalk)
    * [Teoría de la Información](#bayes)

<div id='intro' />

## Introducción

Los laboratorios de estadística computacional (LEC) tienen por objetivo principal analizar datos utilizando técnicas de visualización y evidenciar el comportamiento estocástico de experimentos aleatorios mediante simulaciones computacionales. Las experiencias buscan medir la habilidad de programación en Python y sus librerías, la capacidad de análisis estadístico y la comprensión de documentación, artículos y papers.

Recuerde que los laboratorios tienen una ponderación de 30% en la nota final del ramo.

<div id='reglamento' />

## Reglamento

1. El desarrollo de los laboratorios debe ser en **Python**.
2. El formato de entrega es un **archivo .ipynb**, es decir, un jupyter notebook.
3. El nombre del archivo de entrega del laboratorio $i$ debe seguir el siguiente formato: *lec-i-nombregrupo.ipynb*.
4. Se recomienda seguir las recomendaciones de estilo descritas en [PEP 8](https://www.python.org/dev/peps/pep-0008/) a la hora de programar.
5. El tiempo para la realización de los laboratorios es extenso, por lo que solo se recibirán entregas hasta las 23:59 del día de entrega **a menos que se especifique lo contrario**. Entregas fuera del plazo serán calificadas con nota 0.
6. Antes de entregar su laboratorio verifique su **reproducibilidad**. Jupyter Notebooks con errores a la hora de ejecutarse serán penalizados con descuentos.
7. Solo un integrante por grupo debe realizar la entrega por Aula.

<div id='experience' />

## Experiencia

En el presente laboratorio realizaremos simulaciones computacionales para estudiar distintos experimentos aleatorios. 

### 0. Importación de librerías

In [10]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import scipy.integrate as integrate
from scipy.stats import norm
from scipy.stats import poisson
from scipy.stats import uniform
from scipy.stats import bernoulli

### 1. Funciones útiles

In [2]:
def plot_scatter(x_values, y_values, title=None, x_label=None, y_label=None, height=500, width=800):
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
                    x= x_values,
                    y= y_values,
                    mode='markers',
                    name='markers')
                 )

    fig.update_xaxes(title_text=x_label)
    fig.update_yaxes(title_text=y_label)
    fig.update_layout(title_text=title, title_x=0.5)        
    fig.update_layout(height=height, width=width, template="plotly_dark")

    fig.show(renderer='notebook')

<div id='fundamentals' />

### 2. Fundamentales (10 pts.)

Comenzaremos conociendo las variables aleatorias de SciPy. Gran parte de lo que se desarrollará en esta sección se puede realizar sin problemas en NumPy, cada grupo es libre de utilizar la librería que más sea de su agrado en las siguientes secciones pero en esta **deben responder utilizando SciPy**.

> Se recomienda revisar la documentación:
> 1. [Distribuciones de NumPy](https://numpy.org/doc/1.16/reference/routines.random.html)
> 2. [Distribuciones de SciPy](https://docs.scipy.org/doc/scipy/reference/reference/stats.html)

Supongamos una [variable aleatoria](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bernoulli.html) $X \sim \text{Bernoulli}(0.6)$, en donde $X = 1$ indica que su grupo aprobó el LEC 1 y $X = 0$ que lo reprobó. Para definirla solo debemos hacer lo siguiente:

In [3]:
X = bernoulli(0.6)

Tomemos una muestra de tamaño 1000 y calculemos el promedio muestral:

In [4]:
np.random.seed(21)
sample_mean = np.mean(X.rvs(size=1000))
print(sample_mean)

0.601


> ``np.random.seed`` asegura que la muestra aleatoria sea reproducible, es decir, que siempre se generen los mismos números aleatorios. Puede eliminarla perfectamente. 

Bastante cerca del valor esperado poblacional. Sabemos que la esperanza de una variable aleatoria que sigue una distribución Bernoulli es exactamente $p$, veamos esto en código!

In [5]:
X.mean()

0.6

La varianza también es facilmente obtenible:

In [6]:
X.var()

0.24

El método ``pmf`` (probability mass function) para las variables aleatorias discretas permite calcular la probabilidad de ocurrencia para cada valor en el dominio de la variable aleatoria. Para las variables aleatorias continuas el analogo es el método``pdf`` (probability density function):

> Recuerde que la función de densidad de probabilidad **no indica una probabilidad** para un valor $x$. Para calcular probabilidades con ella se debe integrar sobre el dominio de interés.

In [7]:
X.pmf(0), X.pmf(1)

(0.4, 0.6)

El método ``cdf`` permite calcular la probabilidad acumulada hasta cierto valor:

In [8]:
X.cdf(0), X.cdf(1)

(0.4, 1.0)

El método ``ppf`` permite calcular el valor para el cual la probabilidad acumulada toma un cierto valor:

In [9]:
X.ppf(0.5)

1.0

2.1) Supongamos una variable aleatoria $C \sim N(40, 5)$ que modela la temperatura en Celsius del modelo de computador LEC **(5 pts).**

1. ¿Cuál es la probabilidad de que $C$ tome valores entre 45 [°C] y 50 [°C]?
2. Tome una muestra de 10000 valores desde $C$ y calcule la media muestral.
3. Grafique la distribución de probabilidad de $C$ desde $20$ [°C] hasta $60$ [°C].
4. Grafique la distribución de probabilidad acumulada de $C$ desde $20$ [°C] hasta $60$ [°C].

**Respuesta:**

2.2) Supongamos una variable aleatoria $M \sim \text{Poisson}(70)$ que modela el número de mensajes por minuto enviado desde un computador A hacia otro B mediante una red local **(5 pts.)**

1. ¿Cuál es la probabilidad de que $M$ tome exactamente el valor de la varianza de $M$?
2. ¿Cuál es la probabilidad de que $M$ tome valores entre 60 y 80?
3. Tome una muestra de 10000 valores desde $M$ y calcule la media muestral.
4. Grafique la distribución de probabilidad de $M$ desde $0$ hasta $200$.
5. Grafique la distribución de probabilidad acumulada de $M$ desde $0$ hasta $200$.

**Respuesta:**

<div id='pi' />

### 3. Estimación de $\pi$ (10 pts.)

En nuestra primera simulacion computacional realizaremos una estimacion de $\pi$. Para esto, generaremos $n$ puntos $(x_i, y_i)$ aleatorios en donde $x_i \in [-1,1]$, $y_i \in [-1,1]$. Luego contaremos los puntos que están dentro de la circunferencia de radio 1 y estimaremos $\pi$ en base a la siguiente aproximación:

$$
\frac{\pi r^2}{4r^2} \approx \frac{\Sigma}{n}
$$

Es decir, el ratio entre el area de la circunferencia de radio 1 y el área del cuadrado de lado 2 definido por el rango de los puntos aleatorios debe ser aproximadamente igual al ratio entre la cantidad de puntos $\Sigma$ en la circunferencia y la cantidad total de puntos.

3.1) Defina la función ``estimate_pi`` que estima el valor de $\pi$ utilizando ``n`` puntos **(5 pts.)**

**Respuesta:**

In [2]:
def distance_from_origin(x,y):
    return np.sqrt( ... )

def estimate_pi(n):
    sigma = 0
    
    x,y = np.random.uniform( ... ), np.random.uniform( ... )
    
    for i in range(n):
        if distance_from_origin( ... ) <= 1:
            sigma = ...
            
    pi_estimation = ...
    return pi_estimation    

3.2) Defina la función ``pi_simulation`` que permita visualizar cómo varía la estimación de $\pi$ a medida que el número $n$ de puntos aumenta. ¿Qué indica el gráfico? **(5 pts.)**

**Respuesta:**

In [13]:
def pi_simulation(size=100):
    
    x_values = np.arange( ... )
    y_values = np.array(np.vectorize( ... )( ... ))
    
    plot_scatter(x_values, y_values, title="Estimación Monte Carlo de Pi",
                 x_label="Número de puntos", y_label="Estimación de Pi")

In [1]:
#pi_simulation(size=1000)

<div id='montecarlo' />

### 4. Integración de Monte Carlo (15 pts.)

En nuestra segunda simulación buscaremos aproximar el valor de una integral $\int_{a}^b f(x)dx$ mediante el [método de Monte Carlo](https://www.scratchapixel.com/lessons/mathematics-physics-for-computer-graphics/monte-carlo-methods-in-practice/monte-carlo-integration). La idea es trabajar con el siguiente cálculo de esperanza:

$$ E[f] = \int_{a}^b f(x) \cdot p(x) dx$$

La forma más simple de seguir es considerar $x \sim U(a,b)$, así tenemos:

$$ E[f] = \int_{a}^b f(x) \cdot \frac{1}{b - a} dx$$

De forma que podemos estimar la integral deseada mediante:

$$ \int_{a}^b f(x)dx \approx  E[f] \cdot (b - a) $$

Finalmente, estimaremos $E[f]$ como un promedio empírico utilizando muestras $x_i$ extraídas de $U(a,b)$ :

$$ E[f] \approx \sum_{i = 1}^n f(x_i)$$

4.1) Defina la función ``montecarlo_integration`` que permita estimar el valor de la integral de ``f`` entre ``a`` y ``b`` realizando un sampling de ``n`` puntos desde $U(a, b)$ **(5 pts.)**

**Respuesta:**

In [15]:
def montecarlo_integration(f, a, b, n):
    
    x_values = np.random.uniform( ... )
    y_values = np.vectorize( ... )( ... )
    
    #plot_scatter(x_values, y_values, x_label="x", y_label="y", height=300, width=400)
    
    estimation = ...
    
    return estimation

4.2) Defina la función ``test_montecarlo_accuracy`` que indica cuánto se equivoca (error absoluto) en promedio (cada integral se estima un número ``number_tests`` de veces) su función ``montecarlo_integration`` en la estimación de cada una de las integrales entregadas como parámetro y definidas en la lista ``to_integrate`` en donde cada tupla tiene la forma ($f$,$a$,$b$,valor integral). **(5 pts.)**

In [16]:
to_integrate = [
                    (lambda x: 1/(x**2 + 1), -10, 10, integrate.quad(lambda x: 1/(x**2 + 1), -10, 10)[0]),
                    (np.log, 0.1, 3, integrate.quad(np.log, 0.1, 3)[0]),
                    (lambda x: -1 * x**2, -10, 10, integrate.quad(lambda x: -1 * x**2, -10, 10)[0]),
                    (np.exp, 1, 100, integrate.quad(np.exp, 1, 100)[0])
               ]

In [17]:
def test_montecarlo_accuracy(data, number_tests):
    
    results = list()
    
    for f, a, b, result in data:
        
        results.append( ... )
        
    return results

In [18]:
test_montecarlo_accuracy(to_integrate, 100)

[0.039877611717971974,
 1.3374789275393417,
 1332.5301386618676,
 1.5407673053099325e+42]

4.3) Plantee y fundamente dos modificaciones sencillas que se podrían hacer al método desarrollado y que permitan estimar de forma más precisa las integrales. **(5 pts.)**

**Hint 1:** Las aproximaciones realizas en el planteamiento del método (al inicio de la sección) pueden ser de ayuda.

**Hint 2:** [Ley de los grandes números](https://es.wikipedia.org/wiki/Ley_de_los_grandes_n%C3%BAmeros).

**Respuesta:** 

<div id='randomwalk' />

### 5. Camino aleatorio (25 pts.)

En nuestra tercera simulación analizaremos los [random walk](https://www.britannica.com/science/random-walk): proceso estocástico en el cual objetos que se mueven aleatoriamente se alejan de donde comenzaron. Es importante mencionar que los caminos aleatorios son un ejemplo de [procesos de Markov](https://brilliant.org/wiki/markov-chains/) por lo que cada paso es independiente de los pasados.

Modelaremos la ubicación de un objeto en el tiempo $t$ mediante la siguiente variable aleatoria:

$$
X(t) = \sum_{i = 0}^{t} \phi_i
$$

Donde $\phi_i$ es la **realización de la variable aleatoria** $\phi$ en el paso $i$. Realizaremos la simulación considerando un espacio unidimensional y un camino aleatorio simple donde $P(\phi_i = 1) = P(\phi_i = -1)$, es decir, en cada instante $t$ se puede dar un paso hacia la dirección negativa o hacia la dirección positiva de manera equiprobable. A esta distribución la llamaremos **default**. 

Otra notación análoga e interesante es 

$$X
(t + 1) = X(t) + \phi(t)
$$

La definición de camino aleatorio simple implica que las variables aleatorias $\phi_i$ son **independientes e identicamente distribuidas (IID)**. 

5.1) Defina la función ``random_walk_simulation`` que grafica un camino aleatorio de ``steps`` pasos **(5 pts.)**

**Respuesta:**

In [4]:
def random_walk_simulation(steps):
    
    movements = np.random.choice( ...)
    positions = movements.cumsum()
    
    time = np.arange( ... )
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=..., y=...,mode='lines+markers'))
    fig.update_xaxes(title_text="t")
    fig.update_yaxes(title_text="X(t)")
    fig.update_layout(height=400, width=800, title_text='Random walk', title_x=0.5, template="plotly_dark") #plotly dark, seaborn

    fig.show()

In [2]:
#random_walk_simulation(100)

5.2) Calcule analíticamente la esperanza de $X(t)$, realice su desarrollo en LaTeX **(3 pts.)**

**Respuesta:**

5.3) Defina la función ``empirical_distribution_random_walks`` que permita comprobar empiricamente lo planteado en 4.2 mediante una visualización **(5 pts.)**

**Hint:** El parámetro ``distribution`` será útil en la siguiente pregunta.

**Respuesta:**

In [5]:
def random_walk(steps, distribution = "default"):
    
    if(distribution == "default"):
        
        movements = np.random.choice( ... )
        positions = movements.cumsum() # indexed by time t
        
    return positions
    
def empirical_distribution_random_walks(n,steps, distribution = "default"):
    
    last_positions = [random_walk( ... )[-1] for i in range(n)]
    
    fig = go.Figure(data=[go.Histogram(x=..., histnorm='probability', xbins=dict(start=-steps,end=steps, size=1))])
    fig.update_layout(height=400, width=800, title_text=f'Distribución empírica {distribution} de Random Walks con {steps} pasos', title_x=0.5, template="plotly_dark") #plotly dark, seaborn
    fig.show()

In [3]:
#empirical_distribution_random_walks(10000,10000)

5.4) Visualize la distribución empírica de los Random Walks considerando ahora $\phi = N(0,1)$ y también $\phi = U(-1,1)$. Para esto actualice su función ``random_walk`` **(2 pts.)**|

**Respuesta**

In [6]:
def random_walk(steps, distribution = "default"):
    
    if(distribution == "default"):
        
        movements = np.random.choice( ... )
        positions = movements.cumsum()
        
    if(distribution == "normal"):
        
        movements = np.random.normal( ... )
        positions = movements.cumsum()
        
    if(distribution == "uniform"):

        movements = np.random.uniform( ... )
        positions = movements.cumsum()
    
    return positions

In [4]:
#empirical_distribution_random_walks(10000,10000, distribution = "normal")

In [5]:
#empirical_distribution_random_walks(10000,10000, distribution = "uniform")

5.5) Calcule analiticamente la esperanza y la varianza para cada random walk $\displaystyle X_{\phi_i}(t)$ donde $\phi_1(t) = default, \phi_2(t) = N(0,1)$ y $\phi_3(t) = U(-1,1)$, realice su desarrollo en LaTeX. Luego, explique a qué se debe el comportamiento estocástico visualizado en la pregunta anterior.  **(10 pts.)**

**Hint 1:** Recuerde que un random walk es una suma de variables aleatorias IID.

**Hint 2:** [LOTUS](https://en.wikipedia.org/wiki/Law_of_the_unconscious_statistician)

**Respuesta:**

<div id='bayes' />

### 6. Teoría de la Información (40 pts)

La teoría de la información es una rama de las matemáticas aplicadas que busca **cuantificar cuánta información hay en una señal**. En esta cuarta y última simulación trataremos con el problema que dio origen a este importante campo de estudio: cómo transmitir datos (bits) a través de canales de comunicación ruidosos. 

Consideremos como ejemplo de canal de comunicación un disco duro ruidoso que transmite cada bit incorrectamente con una probabilidad $p$, el esquema de comunicación es el siguiente:

$$
\text{memoria del computador} \rightarrow \text{disco duro} \rightarrow \text{memoria del computador}
$$

Claramente nos interesa hacer $p$ tan pequeño como pueda ser, una solución evidente es la asociada al hardware, es decir, mejorar los componentes del disco duro. La teoría de la información junto a la teoría de la codificación ofrecen una alternativa: aceptar el ruido del canal y **diseñar sistemas de comunicación** que permitan **detectar y corregir los errores introducidos por el canal**. 

Consideremos el siguiente sistema, en donde $t$, $r$ y $\hat{s}$ son diferentes versiones de nuestro mensaje original $s$ .

<img src='system.png' style="width:50%">

> Recuerde la notación pues será de ayuda a la hora de entender los códigos de guía.

El encoder codifica el mensaje original $s$  en un mensaje de transmisión $t$, para esto a $s$ se le agrega cierta redundancia. El decoder recibe el mensaje ruidoso $r$ y utilizando la redundancia añadida por el encoder busca desagregar el mensaje original $s$ del ruido agregado, este mensaje sin ruido es $\hat{s}$ y buscamos que sea $s$.

¿Cuál es la mejor correción de error que podemos conseguir?, let's see!

6.1) Primero que todo debemos representar nuestro mensaje como bits. Defina la función ``string_to_bits`` que recibe un mensaje ``string_message`` y lo representa como un conjunto de bits utilizando el siguiente método: **(3 pts.)**

1. Cada caracter de ``string_message`` se pasa a unicode.
2. Cada unicode asociado a un caracter se pasa a un byte (8 bits).
3. Los bytes asociados a cada unicode se concatenan para formar el mensaje ``bit_message`` a retornar.

**Respuesta:**

In [8]:
def string_to_bits(string_message, show=True):

    bit_message = ""
    data = []

    for char in string_message:

        char_in_unicode = ord(char) 
        unicode_in_binary = str(bin(char_in_unicode))[2:]
        
        if(len(unicode_in_binary) < 8): # fill up to 8 bits
            zeros = ...
            unicode_in_binary = '0' * zeros + unicode_in_binary
        
        bit_message = bit_message + ...
        data.append((char, char_in_unicode, unicode_in_binary))
    
    if(show):
        print(pd.DataFrame(data, columns=["char", "unicode", "binary"])) # handcraft debugger
        
    return bit_message

In [11]:
string_to_bits("Estadística Computacional")

   char  unicode    binary
0     E       69  01000101
1     s      115  01110011
2     t      116  01110100
3     a       97  01100001
4     d      100  01100100
5     í      237  11101101
6     s      115  01110011
7     t      116  01110100
8     i      105  01101001
9     c       99  01100011
10    a       97  01100001
11            32  00100000
12    C       67  01000011
13    o      111  01101111
14    m      109  01101101
15    p      112  01110000
16    u      117  01110101
17    t      116  01110100
18    a       97  01100001
19    c       99  01100011
20    i      105  01101001
21    o      111  01101111
22    n      110  01101110
23    a       97  01100001
24    l      108  01101100


'01000101011100110111010001100001011001001110110101110011011101000110100101100011011000010010000001000011011011110110110101110000011101010111010001100001011000110110100101101111011011100110000101101100'

6.2) Defina la función ``bits_to_string`` que permite recuperar el mensaje originial desde ``bit_message``. **(2 pts)**.


**Respuesta:**

In [23]:
def bits_to_string(bit_message):
    
    unicodes_in_binary = [ bit_message[i:i+8] for i in range(0, len(bit_message), 8) ]
    unicodes = ...
    strings = list(map(chr, unicodes))
    
    return ''.join(strings)

**[checkpoint]** Ejecute las siguientes dos celdas para verificar sus funciones! 

In [18]:
def everything_ok(string_message, show=False):
    bits = string_to_bits(string_message, show=show)
    return bits_to_string(bits) == string_message

In [22]:
everything_ok("Ánimo, ya están pronto a finalizar su LEC 2 :')")

True

Modelaremos el ruido como un **vector de variables aleatorias** $\text{Bernoulli}(p)$, de esta forma el bit en la posicion $i$ del mensaje $t$ será modificado si el número en la posición $i$ del vector de ruido es $1$ (este evento pasa con probabilidad $p$).

6.3) Defina la función ``noisy_channel`` que modela un canal de comunicación en donde cada bit de ``t`` se transmite incorrectamente con una probabilidad ``p`` **(5 pts.)**

**Respuesta:**

In [30]:
def noisy_channel(t, p): # p is probability of error !!
    
    t_array = np.fromiter(t, dtype=int) 
    noise = bernoulli.rvs( ... )
    t_masked = t_array ^ noise # XOR operator
    r = ''.join( ... ) 
    
    return r 

Veamos cuánto se corrompe un mensaje al pasarlo **sin ningún tipo de seguridad** (sin encoder aún) por un **canal de comunicación ruidoso con una probabilidad muy baja de corrupción**: 

In [31]:
s = string_to_bits("Hola :'), este es un mensaje de prueba! Aprovecho de recordarles que comenten adecuadamente su código. ")
r = noisy_channel(s, 0.01)
s_prime = bits_to_string(r) 
print(s_prime)

Hola :), este es un mensaje de prueba! Aprovechk de bec/rdarlus que comenten adecuadamånte su códagm. 


Nuestro mensaje se corrompe facilmente aún cuando la probabilidad de que cada bit se transmita erroneamente sea $p = 0.01$. Es hora de mejorar la transmisión de datos utilizando un encoder y un decoder.

6.4) Defina la función ``encoder`` que repite ``redundancy`` veces cada bit del mensaje ``s`` **(2 pts.)**

**Respuesta:**

In [32]:
def encoder(s, redundancy):
    t = [ ...  for bit in s]# repeat redundancy times every bit
    return ''.join(t)

6.5) Defina la función ``decoder`` que recibe un mensaje ``r`` **ruidoso y codificado** y lo decodifica utilizando el siguiente método en donde se asume $p < 0.5$: **(5 pts.)**

1. El mensaje ``r`` se agrupa cada ``redundancy`` bits seguidos.
2. Cada grupo de ``redundancy`` bits realiza una votación democrática para elegir un bit que los represente. Este bit será 1 si en el grupo hay más bits con el valor 1, de lo contrario, será 0. Notar que el caso de igualdad no es de interés, **la redundancia introducida por el decoder debe ser impar**.
3. Los bits que reprensentan cada grupo de ``redundancy`` bits se concatenan y conforman el mensaje decodificado.

In [24]:
def democratic_vote(sequence, n):
    redundancy = len(sequence)
    
    if(redundancy % 2 != 0):
        ones = 0
        zeros = 0
        for bit in sequence:
            if(bit == '1'):
                ones = ones + 1
            else:
                zeros = zeros + 1
        if(zeros > ones):
            return '0'
        else:
            return '1'  
    else:
        raise ValueError(f"Redundancy level should be odd")
        
    
def decoder(r, redundancy):
    
    r_grouped = ...
    s_prime = ...
    
    return ''.join(s_prime)

6.6) Defina la función ``communication_test`` que permite simular el flujo de una comunicación  a través de un canal ruidoso con probabilidad ``p`` de corromper cada bit. Si el parámetro ``encode`` es ``True`` se debe codificar el mensaje con un un nivel de redundancia ``redundancy``, de lo contrario no se debe codificar. **(3 pts.)**

**Respuesta:**

In [49]:
def communication_test(string_message, p, encode=False, redundancy=1001):
    
    s = string_to_bits(string_message)
    
    if(encode):
        t = encoder( ... )
        r = noisy_channel( ... )
        s_prime = bits_to_string( ... )
        
    else:
        r = noisy_channel( ... )
        s_prime = bits_to_string( ... )
        
    return s_prime

In [50]:
message = "Veamos si hay alguna mejora después de tanto esfuerzo :-)"

print(f"Sin codificar: {communication_test(message, 0.45)}")
print(f"Con codificar: {communication_test(message, 0.45, encode=True)}")

 °CPëä):Nñ,êÛ½#`Ë´`¶×ÏÝÔ©L¿h¤©éM®åd¾9èwÊfrµ
Con codificar: Veamos si hay alguna mejora después de tanto ecfuerzo :-)


Mucho mejor la versión codificada !

6.7) Explique sucintamente por qué el utilizar un encoder y decoder la probabilidad de que el mensaje se transmita de manera correcta aumenta **(2 pts)**.

**Respuesta:**

6.8) Defina la función ``probability_bit_corruption`` que aproxima mediante ``n`` simulaciones la probabilidad de que un **bit del mensaje original** se transmita de manera errónea a través de un canal ruidoso con probabilidad ``p`` de corrupción cuando se utiliza una redundancia ``redundancy`` y la función ``democratic_vote`` **(5 pts.)**

**Respuesta:**

In [52]:
def probability_bit_corruption(p, redundancy, n):
    right = 0
    wrong = 0
    s = '0' # or '1'
    
    for i in range(n):
        t = ...
        r = ...
        if(s == decoder(r, redundancy)):
            right = ...
        else:
            wrong = ...
            
    return ...

La siguiente función permite verificar su función ``probability_bit_corruption`` y también será de ayuda para la próxima pregunta.

In [25]:
def plot_empirical_bit_corruption_distribution(n, p):
    x_values = np.arange(1, 500, 2) # max redundancy
    y_values = list(map(lambda redundancy: probability_bit_corruption(p, redundancy, n), x_values))
    
    plot_scatter(x_values, y_values, x_label="Encoder redundancy", y_label="corruption probability",
                 title="Empirical bit corruption distribution",height= 400, width=700)

In [6]:
#plot_empirical_bit_corruption_distribution(100, 0.4) # hard work

6.9) ¿Qué pasa cuando $p > 0.5$?, ¿por qué? **(8 pts.)**

**Respuesta:**

6.10) El decoder busca maximizar $P(s|r)$, donde $s$ es un bit y $r$ como de costumbre el bit $s$ codificado por el encoder y modificado por el ruido del canal de transmisión. Interprete $P(s|r)$ y su maximización **(5 pts.)**

**Respuesta:**

## The end of the road (or the beginning of a new one) 

In [1]:
print("""
Felicitaciones ! Ya completaste el segundo laboratorio de Estadística Computacional :') 
""")


Felicitaciones ! Ya completaste el segundo laboratorio de Estadística Computacional :') 

